In [1084]:
import numpy as np

def joint_similarity(joint_x,joint_y):
    #joint_x
    x_coords_x = [x[0] for x in joint_x]
    y_coords_x = [y[1] for y in joint_x] 
    
    #считаем длины сегментов joint_x 
    left_length_x = ((x_coords_x[1] - x_coords_x[0])**2 + (y_coords_x[1] - y_coords_x[0])**2)**0.5
    right_lentgh_x = ((x_coords_x[1] - x_coords_x[2])**2 + (y_coords_x[1] - y_coords_x[2])**2)**0.5
    
    #отношение длин сегментов joint_x
    gamma_x = left_length_x/right_lentgh_x
    
    #угол joint_x
    point_a_x = [x_coords_x[0]-x_coords_x[1], y_coords_x[0]-y_coords_x[1]]
    point_b_x = [x_coords_x[2]-x_coords_x[1], y_coords_x[2]-y_coords_x[1]]
    ang_a_x = np.arctan2(*point_a_x[::-1])
    ang_b_x = np.arctan2(*point_b_x[::-1])
    angle_x = np.rad2deg((ang_b_x - ang_a_x) % (2 * np.pi))*np.pi/180 
    
    #joint_y
    x_coords_y = [x[0] for x in joint_y]
    y_coords_y = [y[1] for y in joint_y] 
    
    #считаем длины сегментов joint_y 
    left_length_y = ((x_coords_y[1] - x_coords_y[0])**2 + (y_coords_y[1] - y_coords_y[0])**2)**0.5
    right_lentgh_y = ((x_coords_y[1] - x_coords_y[2])**2 + (y_coords_y[1] - y_coords_y[2])**2)**0.5
    
    #отношение длин сегментов joint_y
    gamma_y = left_length_y/right_lentgh_y
    
    #угол joint_y
    point_a_y = [x_coords_y[0]-x_coords_y[1], y_coords_y[0]-y_coords_y[1]]
    point_b_y = [x_coords_y[2]-x_coords_y[1], y_coords_y[2]-y_coords_y[1]]
    ang_a_y = np.arctan2(*point_a_y[::-1])
    ang_b_y = np.arctan2(*point_b_y[::-1])
    angle_y = np.rad2deg((ang_b_y - ang_a_y) % (2 * np.pi))*np.pi/180 
    
    segment_length_ratio = math.exp(-0.5*(1-min(gamma_x/gamma_y, gamma_y/gamma_x)))
    
    segment_angle_ratio = math.exp(-2*abs(angle_x-angle_y))
    
    
    return segment_length_ratio * segment_angle_ratio


#функция, округляющая с необходимой точностью chain, но она оказалась не нужна ):
def to_corr_chain(chain): 
    
    #делаем loop для chain, добавляя в конец первую точку, а в начало последнюю
    loop_chain=[]
    loop_chain.append(chain[len(chain)-1])
    for j in range(len(chain)):
        loop_chain.append(chain[j])
    loop_chain.append(chain[0])
    x_coords = [x[0] for x in loop_chain]
    y_coords = [y[1] for y in loop_chain] 
    
    lengths_ratios=[]
    angles=[]
    
    #считаем углы джоинтов и отношение длин сегментов, теперь каждый джоинт - это пара (отношение длинн, угол)
    for j in range(len(loop_chain)-2):
        
        left_length = ((x_coords[j] - x_coords[j+1])**2 + (y_coords[j] - y_coords[j+1])**2)**0.5
        right_lentgh = ((x_coords[j+2] - x_coords[j+1])**2 + (y_coords[j+2] - y_coords[j+1])**2)**0.5        
        gamma_x = left_length/right_lentgh
        lengths_ratios.append(left_length/right_lentgh) 
        
        
        
        
        point_a = [x_coords[j]-x_coords[j+1], y_coords[j]-y_coords[j+1]]
        point_b = [x_coords[j+2]-x_coords[j+1], y_coords[j+2]-y_coords[j+1]]
        ang_a = np.arctan2(point_a[1],point_a[0])
        ang_b = np.arctan2(point_b[1], point_b[0])
        angle = np.rad2deg((ang_b - ang_a) % (2 * np.pi))
        angles.append(angle)

    #chain_pairs = np.column_stack((lengths_ratios,angles))
    #print angles 
    #print lengths_ratios
#каждый chain теперь это последовательность пар гамм(отношение длин сегментов джоинта) и тет(угол джоинта),
#для того чтобы был joint_match нам нужно переделать переделать отношения длин, 
#которые в принципе вещественные -> в натуральные, а углы каждый из которых от [0,2*pi] в целые 
#в итоге с необходимой точностью джоинт [1.0, 154.45] и джоинт [1.1, 150.789] перейдут, например в джоинт [30, 78] 
    
    
#следующий цикл с необходимой точностью ставит в соответствие действительному числу натуральное(число ноль пусть тоже натуральное). 
#например с точностью 0.5 числа 0.51 0.9, 0.(98) - станут равными единице, 0.49 - станет нулем. 
   
    accur_angles=[] #-массив округленных с необходимой точностью значений углов(тет)
    accur_lengths_ratios=[] #-массив округленных с необходимой точностью гамм
    for j in range(len(chain)): 
        angle_accuracy = 0.1
        gamma_accuracy = 0.1
        accur_angles.append(int(angles[j]/angle_accuracy)) 
        accur_lengths_ratios.append(int(lengths_ratios[j]/gamma_accuracy))
        accur_chain_pairs = np.column_stack((accur_lengths_ratios,accur_angles))
    #print accur_angles 
    #print accur_lengths_ratios
    return accur_chain_pairs
    
#необходимо создать три матрицы:
# 1) матрицу "сходства"(sim_matrix) где ячейка [i,j] будет соответстовать значению сходства(S_jnt) между i-ым и j-ым джоинтами 
# 2) матрицу "штрафов"(pen_matrix) где ячейка [i, j] будет соответсвовать значению штрафа в случае skipping 
# 3) матрицу Smith-Waterman(S_W_matrix) - матрица, получаемая из двух предыдущих матриц 


def sim_matrix(chain1, chain2):
#в этой функции необходимо считать similarity между каждым джоинтом по принципу "каждый с каждым",
#для этого воспользуемся функцией joint_similarity(J_x,J_y). Ее аргументами являются "тройки" пар, поэтому
#разобьем каждый из chain на последовательные тройки пар с интервалом в одну пару. 
#например если chain - [(1,1),(2,2),(3,3)] - то получим массив из троек - ([[(1,1),(2,2),(3,3)],[(2,2),(3,3),(1,1)]),[]
    
    #looping chain1
    loop_chain1=[]
    loop_chain1.append(chain1[len(chain1)-1])
    for j in range(len(chain1)):
        loop_chain1.append(chain1[j])
    loop_chain1.append(chain1[0])
    
    #массив джоинтов у chain1
    seq_of_joints1 = []
    for j in range(len(loop_chain1)-2):
        seq_of_joints1.append((loop_chain1[j], loop_chain1[j+1],loop_chain1[j+2]))
    
    
    #looping chain1
    loop_chain2=[]
    loop_chain2.append(chain2[len(chain2)-1])
    for j in range(len(chain2)):
        loop_chain2.append(chain2[j])
    loop_chain2.append(chain2[0])
    
    #массив джоинтов у chain2
    seq_of_joints2 = []
    for j in range(len(loop_chain2)-2):
        seq_of_joints2.append((loop_chain2[j], loop_chain2[j+1],loop_chain2[j+2]))
    
    H = np.zeros((len(chain1) + 1, len(chain2) + 1))
    
    for i in range(len(chain1)):
        for j in range(len(chain2)):
            H[i+1,j+1]=joint_similarity(seq_of_joints1[i], seq_of_joints2[j])
    return H 

#матрица штрафов для chain, пусть lambda_skc = 1
def pen_matrix(chain):
    
    lambda_skc = 1.0 #чувствительность от потери джоинта в длинах его сегмента
    loop_chain=[]
    loop_chain.append(chain[len(chain)-1])
    for j in range(len(chain)):
        loop_chain.append(chain[j])
    loop_chain.append(chain[0])
    x_coords = [x[0] for x in loop_chain]
    y_coords = [y[1] for y in loop_chain] 
    
    lengths_ratios=[]
    angles=[]
    penalties = []
    for j in range(len(loop_chain)-2):
        left_length = ((x_coords[j] - x_coords[j+1])**2 + (y_coords[j] - y_coords[j+1])**2)**0.5
        right_lentgh = ((x_coords[j+2] - x_coords[j+1])**2 + (y_coords[j+2] - y_coords[j+1])**2)**0.5        
        point_a = [x_coords[j]-x_coords[j+1], y_coords[j]-y_coords[j+1]]
        point_b = [x_coords[j+2]-x_coords[j+1], y_coords[j+2]-y_coords[j+1]]
        ang_a = np.arctan2(point_a[1],point_a[0])
        ang_b = np.arctan2(point_b[1], point_b[0])
        angle = np.rad2deg((ang_b - ang_a) % (2 * np.pi))
        pen = 1 - math.exp(-abs(np.pi - angle)) + lambda_skc*(left_length+right_lentgh)/2
        penalties.append(pen)
    return penalties
    
def distance(chain1, chain2): 
        H = np.zeros((len(chain1) + 1, len(chain2) + 1))
        #corr_chain1 = to_corr_chain(chain1)
        #corr_chain2 = to_corr_chain(chain2)
        s_matrix  = sim_matrix(chain1, chain2)
        
        p_matrix1 = pen_matrix(chain1)
        p_matrix2 = pen_matrix(chain2)
        
        max_score = 0 
        max_pos   = None
        for i in range(len(chain1)):
            for j in range(len(chain2)):
                H[i+1,j+1] = max(H[i-1,j-1]+s_matrix[i+1,j+1],H[i,j+1]- p_matrix1[i],H[i+1,j]- p_matrix2[j])
                if  H[i+1,j+1] > max_score:
                    max_score = H[i+1,j+1]
                    max_pos   = (i+1, j+1)
        print H
        #traceback
        chains_similarity = max_score
        next_value = 0
        candidates=[]
        #i=0
        #j=0
        while True:
            
            next_value = max(H[max_pos[0], max_pos[1]-1], H[max_pos[0]-1, max_pos[1]-1], H[max_pos[0]-1, max_pos[1]])
            candidates = np.asarray([H[max_pos[0], max_pos[1]-1], H[max_pos[0]-1, max_pos[1]-1], H[max_pos[0]-1, max_pos[1]]])
            
            print "candidates", candidates,'!', "next_value",next_value,'!'
            
            if candidates.argmax() == 0:
                max_pos = [max_pos[0], max_pos[1]-1]
            if candidates.argmax() == 1:
                max_pos = [max_pos[0]-1, max_pos[1]-1]
            if candidates.argmax() == 2:
                 max_pos = [max_pos[0]-1, max_pos[1]]
            if max_pos[1] == 0:
                break
            if max_pos[0] == 0:
                break
            
            #print "новая max_pos", max_pos, "max_pos[0]-", max_pos[0],'max_pos[1]-', max_pos[1]
            chains_similarity = chains_similarity + next_value 
            #print "chains_similarity", chains_similarity, "!"
        chains_similarity = chains_similarity/min(len(chain1)+1,len(chain2)+1)
        return chains_similarity
        







In [1085]:
to_corr_chain([(0.0,1.0),(1.0,0.0),(1.0,1.0)])

array([[   7, 3150],
       [  14, 3150],
       [  10, 2700]])

In [1086]:
sim_matrix([(0.0,0.0),(1.0,0.0),(1.0,1.0)],[(0.0,0.0),(1.0,1.0),(0.0,1.0)])

array([[ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.77880078,  1.        ,  0.17956053],
       [ 0.        ,  0.17956053,  0.17956053,  1.        ],
       [ 0.        ,  1.        ,  0.77880078,  0.17956053]])

In [1087]:
pen_matrix([(0.0,0.0),(1.0,0.0),(1.0,1.0)])

[2.2071067811865475, 2.0, 2.2071067811865475]

In [1089]:
distance([(11.0,0.0),(2.0,0.0),(19.0,94.0)],[(18.0,0.0),(6.0,0.0),(19.0,95.0)])

[[ 0.          0.          0.          0.        ]
 [ 0.          0.76253523  0.39226062  0.02982789]
 [ 0.          0.4177233   0.81105098  0.05048194]
 [ 0.          0.06284709  0.04416739  1.70003312]]
candidates [ 0.04416739  0.81105098  0.05048194] ! next_value 0.811050979481 !
candidates [ 0.4177233   0.76253523  0.39226062] ! next_value 0.762535230486 !
candidates [ 0.  0.  0.] ! next_value 0.0 !


0.81840483146654974

In [672]:
max(3,4,5)

5